In [1]:
# Node Classification with DGL
# --> node classification using GNN
# ----> load DGL-provided dataset
# ----> Build a GNN model with DGL-provided neural network modules
# ----> Train and evaluate a GNN model for node classification 

In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/parton15/anaconda3/envs/graphNN/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load Cora Dataset

import dgl.data
dataset = dgl.data.CoraGraphDataset()
print("Number of Categories:", dataset.num_classes)

Extracting file to /home/parton15/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of Categories: 7


In [7]:
g = dataset[0]
#print(g)

In [8]:
print("Node Features")
print(g.ndata)

Node Features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}


In [9]:
print("Edge Features")
print(g.edata)

Edge Features
{}


In [10]:
#### Defining Graph Convolutional Network (GCN)

from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [11]:
### Training the GCN

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.148 (best 0.148), test acc: 0.147 (best 0.147)
In epoch 5, loss: 1.889, val acc: 0.574 (best 0.574), test acc: 0.571 (best 0.571)
In epoch 10, loss: 1.806, val acc: 0.554 (best 0.592), test acc: 0.587 (best 0.599)
In epoch 15, loss: 1.699, val acc: 0.600 (best 0.600), test acc: 0.621 (best 0.621)
In epoch 20, loss: 1.570, val acc: 0.652 (best 0.652), test acc: 0.666 (best 0.666)
In epoch 25, loss: 1.419, val acc: 0.666 (best 0.666), test acc: 0.678 (best 0.678)
In epoch 30, loss: 1.253, val acc: 0.694 (best 0.694), test acc: 0.698 (best 0.698)
In epoch 35, loss: 1.080, val acc: 0.714 (best 0.714), test acc: 0.713 (best 0.713)
In epoch 40, loss: 0.910, val acc: 0.728 (best 0.728), test acc: 0.731 (best 0.729)
In epoch 45, loss: 0.751, val acc: 0.748 (best 0.748), test acc: 0.750 (best 0.750)
In epoch 50, loss: 0.611, val acc: 0.756 (best 0.756), test acc: 0.759 (best 0.759)
In epoch 55, loss: 0.492, val acc: 0.766 (best 0.766), test acc: 0.761 (best 0